# 📘 Explorador y gestor de la base de datos Everest
Este notebook permite crear la base de datos, agregar usuarios y autenticar credenciales mediante funciones reutilizables.

In [1]:
import sqlite3
import hashlib
from datetime import datetime
from typing import Optional
import pandas as pd


In [2]:
def conectar_db(DB_PATH):
    try:
        conn = sqlite3.connect(DB_PATH)
        print(f"Conectado a la base de datos")
        return conn
    except sqlite3.Error as e:
        print(f"Error al conectar con la base de datos: {e}")
        return None


In [3]:

def crear_tablas(DB_PATH):
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password TEXT NOT NULL,
        role TEXT NOT NULL
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS progress (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        level INTEGER,
        timestamp TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS test_results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        test_id TEXT,
        score INTEGER,
        timestamp TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS avatar_profiles (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        skin TEXT,
        outfit TEXT,
        equipment TEXT,
        -- Columnas para las habilidades del avatar y su símil laboral
        fuerza INTEGER DEFAULT 0,
        agilidad INTEGER DEFAULT 0,
        equilibrio INTEGER DEFAULT 0,
        resistencia_clima INTEGER DEFAULT 0,
        vision INTEGER DEFAULT 0,
        agarre INTEGER DEFAULT 0,
        capacidad_carga INTEGER DEFAULT 0,
        habilidad_equipo INTEGER DEFAULT 0,
        comunicacion INTEGER DEFAULT 0,
        orientacion INTEGER DEFAULT 0,
        determinacion INTEGER DEFAULT 0,
        sentido_roca INTEGER DEFAULT 0,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS medals (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        description TEXT,
        test_id TEXT,
        awarded_on TEXT,
        FOREIGN KEY(user_id) REFERENCES users(id)
    );
    """)

    conn.commit()
    conn.close()
    print("✅ Tablas creadas o ya existentes.")

# Ejemplo de uso (asegúrate de que DB_PATH apunte a tu base de datos)
if __name__ == "__main__":
    DB_PATH = 'your_database.db' # Cambia esto a la ruta de tu base de datos
    crear_tablas(DB_PATH)

Conectado a la base de datos
✅ Tablas creadas o ya existentes.


In [4]:
def agregar_usuario(DB_PATH,username: str, password: str, role: str = "player"):
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()
    hashed_pw = hashlib.sha256(password.encode()).hexdigest()

    try:
        cursor.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                       (username, hashed_pw, role))
        conn.commit()
        print(f"✅ Usuario '{username}' agregado con rol '{role}'")
    except sqlite3.IntegrityError:
        print(f"⚠️ El usuario '{username}' ya existe.")
    finally:
        conn.close()


In [5]:
def autenticar_usuario(DB_PATH,username: str, password: str) -> Optional[dict]:
    conn = conectar_db(DB_PATH)
    cursor = conn.cursor()
    hashed_pw = hashlib.sha256(password.encode()).hexdigest()

    cursor.execute("SELECT id, role FROM users WHERE username = ? AND password = ?",
                   (username, hashed_pw))
    user = cursor.fetchone()
    conn.close()

    if user:
        return {"status": "success", "user_id": user[0], "role": user[1]}
    else:
        return None


In [6]:

def listar_tablas(DB_PATH):
    conn = conectar_db(DB_PATH)
    if conn is None:
        raise RuntimeError("No se pudo conectar a la base de datos.")

    try:
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        tablas_df = pd.read_sql(query, conn)
        print("📋 Tablas encontradas:")
        # display(tablas_df)
        return tablas_df
    except Exception as e:
        print(f"❌ Error al listar tablas: {e}")
    finally:
        conn.close()

In [7]:
import yaml
from pathlib import Path

def cargar_configuracion(ruta_config=None):
    """
    Carga un archivo config.yml. Si no se pasa ruta_config, se solicita por input.
    Retorna un diccionario con la configuración cargada.
    """
    if ruta_config is None:
        ruta_config = input("Ingrese la ruta del archivo config.yml: ").strip()

    ruta = Path(ruta_config)
    if not ruta.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {ruta_config}")

    with open(ruta, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    print(f"Configuración cargada")
    return config


In [ ]:
def inicializar_perfiles_avatar_para_nuevos_usuarios(db_path):
    """
    Crea un perfil de avatar por defecto para cada usuario que no tenga uno en avatar_profiles.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()

    cursor.execute("""
        INSERT INTO avatar_profiles (username)
        SELECT u.username
        FROM users u
        LEFT JOIN avatar_profiles a ON u.username = a.username
        WHERE a.username IS NULL
    """)
    conn.commit()
    conn.close()
    print("✅ Perfiles de avatar inicializados para usuarios sin perfil.")


def actualizar_habilidad_avatar(db_path, user_id, habilidad, valor_cambio):
    """
    Actualiza el valor de una habilidad específica del avatar de un usuario.
    
    Args:
        db_path (str): Ruta al archivo de la base de datos.
        user_id (int): ID del usuario cuyo avatar se va a actualizar.
        habilidad (str): Nombre de la columna de la habilidad a actualizar (ej. 'fuerza_puntos').
        valor_cambio (int): Cantidad a sumar o restar al valor actual de la habilidad.
    
    Returns:
        bool: True si la actualización fue exitosa, False en caso contrario.
    """
    if not habilidad.endswith('_puntos'):
        print("⚠️ Nombre de habilidad inválido. Debe terminar en '_puntos'.")
        return False
        
    conn = conectar_db(db_path)
    cursor = conn.cursor()
    
    try:
        # Asegurarse de que el perfil exista antes de intentar actualizar
        cursor.execute("SELECT id FROM avatar_profiles WHERE user_id = ?", (user_id,))
        if cursor.fetchone() is None:
            print(f"⚠️ No existe un perfil de avatar para el usuario con ID {user_id}.")
            conn.close()
            return False

        # Prevenir inyección SQL asegurando que el nombre de la columna sea válido
        valid_habilidades = [
            'fuerza_puntos', 'agilidad_puntos', 'equilibrio_puntos',
            'resistencia_clima_puntos', 'vision_puntos', 'agarre_puntos',
            'capacidad_carga_puntos', 'habilidad_equipo_puntos',
            'comunicacion_puntos', 'orientacion_puntos', 'determinacion_puntos',
            'sentido_roca_puntos'
        ]
        if habilidad not in valid_habilidades:
            print(f"⚠️ La habilidad '{habilidad}' no es una columna válida para actualizar.")
            conn.close()
            return False

        # Actualizar la habilidad
        query = f"UPDATE avatar_profiles SET {habilidad} = {habilidad} + ? WHERE user_id = ?"
        cursor.execute(query, (valor_cambio, user_id))
        conn.commit()
        print(f"✅ Habilidad '{habilidad}' del usuario {user_id} actualizada por {valor_cambio}.")
        return True
    except sqlite3.Error as e:
        print(f"❌ Error al actualizar la habilidad: {e}")
        return False
    finally:
        conn.close()

def obtener_perfil_avatar(db_path, user_id):
    """
    Obtiene el perfil completo de avatar para un usuario.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM avatar_profiles WHERE user_id = ?", (user_id,))
    perfil = cursor.fetchone()
    conn.close()
    
    if perfil:
        # Convertir las cadenas JSON de vuelta a listas
        # Asumiendo que el orden de las columnas es conocido para acceder por índice
        column_names = [description[0] for description in cursor.description]
        perfil_dict = dict(zip(column_names, perfil))
        
        if 'skin' in perfil_dict and perfil_dict['skin']:
            perfil_dict['skin'] = json.loads(perfil_dict['skin'])
        else:
            perfil_dict['skin'] = []
            
        if 'outfit' in perfil_dict and perfil_dict['outfit']:
            perfil_dict['outfit'] = json.loads(perfil_dict['outfit'])
        else:
            perfil_dict['outfit'] = []

        if 'equipment' in perfil_dict and perfil_dict['equipment']:
            perfil_dict['equipment'] = json.loads(perfil_dict['equipment'])
        else:
            perfil_dict['equipment'] = []
            
        return perfil_dict
    return None


## 🧪 Crear tablas y probar flujo completo

In [9]:
'C:/Users/anmmu/OneDrive/Proyectos/PSYCH_CLIMBERS/PsychTest-Gaming-Hub/config.yml'
config = cargar_configuracion() 

Configuración cargada


In [10]:

ENV_DIR = config["paths"]["env_dir"]
DB_PATH = ENV_DIR+ config["paths"]["db_file"]

In [11]:
crear_tablas(DB_PATH)

Conectado a la base de datos
✅ Tablas creadas o ya existentes.


In [12]:
listar_tablas(DB_PATH)

Conectado a la base de datos
📋 Tablas encontradas:


,name
0,users
1,sqlite_sequence
2,progress
3,test_results
4,avatar_profiles
5,medals


In [13]:

conn = sqlite3.connect(DB_PATH)
df_users = pd.read_sql("SELECT * FROM users", conn)
display(df_users)
conn.close()


,id,username,password,role


In [14]:
# Agregar un usuario de prueba
# agregar_usuario("demo_user", "1234", "player")

lista_usarios = [
    {'username' : "admin", 'password' : "987654321",'role' : "administrator"},
    {'username' : "Oscar", 'password' : "123456",'role' : "player"},
    {'username' : "Ana", 'password' : "123456",'role' : "player"},
    {'username' : "Andres", 'password' : "123456",'role' : "player"},
    {'username' : "Diana", 'password' : "123456",'role' : "player"},
    {'username' : "Maria", 'password' : "123456",'role' : "player"},
    {'username' : "sico_001", 'password' : "123456",'role' : "moderador"},
    {'username' : "sico_002", 'password' : "123456",'role' : "moderador"},
    {'username' : "invitado", 'password' : "123456",'role' : "player"},
    {'username' : "invitada", 'password' : "123456",'role' : "player"}
]
for usuario_nuevo in lista_usarios:
    agregar_usuario(DB_PATH,usuario_nuevo['username'], usuario_nuevo['password'], usuario_nuevo['role'])
    # print(usuario_nuevo['username'])
    print(usuario_nuevo)


Conectado a la base de datos
✅ Usuario 'admin' agregado con rol 'administrator'
{'username': 'admin', 'password': '987654321', 'role': 'administrator'}
Conectado a la base de datos
✅ Usuario 'Oscar' agregado con rol 'player'
{'username': 'Oscar', 'password': '123456', 'role': 'player'}
Conectado a la base de datos
✅ Usuario 'Ana' agregado con rol 'player'
{'username': 'Ana', 'password': '123456', 'role': 'player'}
Conectado a la base de datos
✅ Usuario 'Andres' agregado con rol 'player'
{'username': 'Andres', 'password': '123456', 'role': 'player'}
Conectado a la base de datos
✅ Usuario 'Diana' agregado con rol 'player'
{'username': 'Diana', 'password': '123456', 'role': 'player'}
Conectado a la base de datos
✅ Usuario 'Maria' agregado con rol 'player'
{'username': 'Maria', 'password': '123456', 'role': 'player'}
Conectado a la base de datos
✅ Usuario 'sico_001' agregado con rol 'moderador'
{'username': 'sico_001', 'password': '123456', 'role': 'moderador'}
Conectado a la base de datos

In [15]:

conn = sqlite3.connect(DB_PATH)
df_users = pd.read_sql("SELECT * FROM users", conn)
display(df_users)
conn.close()


,id,username,password,role
0,1,admin,8a9bcf1e51e812d0af8465a8dbcc9f741064bf0af3b3d0...,administrator
1,2,Oscar,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
2,3,Ana,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
3,4,Andres,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
4,5,Diana,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
5,6,Maria,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
6,7,sico_001,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,moderador
7,8,sico_002,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,moderador
8,9,invitado,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player
9,10,invitada,8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3...,player


In [24]:
# 3. Inicializar perfiles de avatar para los usuarios existentes
inicializar_perfiles_avatar_para_nuevos_usuarios(DB_PATH)

Conectado a la base de datos
✅ Perfiles de avatar inicializados para usuarios sin perfil.


In [25]:

conn = sqlite3.connect(DB_PATH)
df_avatares = pd.read_sql("SELECT * FROM avatar_profiles", conn)
display(df_avatares)
conn.close()


,id,username,fuerza,agilidad,equilibrio,resistencia_frio,vision,agarre,capacidad_carga,habilidad_equipo,comunicacion,orientacion,determinacion,sentido_roca
0,1,3,None,None,None,None,None,None,None,None,None,None,None,None
1,2,4,None,None,None,None,None,None,None,None,None,None,None,None
2,3,5,None,None,None,None,None,None,None,None,None,None,None,None
3,4,6,None,None,None,None,None,None,None,None,None,None,None,None
4,5,2,None,None,None,None,None,None,None,None,None,None,None,None
5,6,1,None,None,None,None,None,None,None,None,None,None,None,None
6,7,10,None,None,None,None,None,None,None,None,None,None,None,None
7,8,9,None,None,None,None,None,None,None,None,None,None,None,None
8,9,7,None,None,None,None,None,None,None,None,None,None,None,None
9,10,8,None,None,None,None,None,None,None,None,None,None,None,None


In [26]:
df_users.columns

Index(['id', 'username', 'password', 'role'], dtype='object')

In [ ]:
def crear_o_reemplazar_perfil_avatar(db_path, username):
    """
    Crea un nuevo perfil de avatar para el usuario dado.
    Si ya existe, lo elimina y lo reemplaza con valores por defecto.
    """
    conn = conectar_db(db_path)
    cursor = conn.cursor()

    # Elimina el perfil si ya existe
    cursor.execute("DELETE FROM avatar_profiles WHERE username = ?", (username,))

    # Inserta un nuevo perfil con valores por defecto (por ejemplo: 50 en todo)
    cursor.execute("""
        INSERT INTO avatar_profiles (
            username, fuerza, agilidad, equilibrio, resistencia_frio,
            vision, agarre, capacidad_carga, habilidad_equipo,
            comunicacion, orientacion, determinacion, sentido_roca
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (username, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50))

    conn.commit()
    conn.close()
    print(f"✅ Perfil de avatar creado o reemplazado para '{username}'.")


In [ ]:
crear_o_reemplazar_perfil_avatar(DB_PATH, "Andres")

Conectado a la base de datos


OperationalError: database is locked